### Imports and Utility-Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

%run ./library/traffic_simulation.py
%run ../utils/helper.py
%run /workspaces/jupyterlite/content/pytroch/arrays_and_matricies.py
%run /workspaces/jupyterlite/content/pytorch-physics/pytorch-traffic-simulation.py

In [ ]:
def draw_lines_based_on_points_on_plot(points, ax=None, line_color='blue', line_size=0.5):
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(points[:, 0], points[:, 1], color=line_color, linewidth=line_size)

    return ax

def draw_points_on_plot(point_sequence, ax, line_color, point_size=1):
    x, y = zip(*point_sequence)
    ax.scatter(x, y, color=line_color, s=point_size)
    return ax

In [ ]:
def visualize_line_change_by_theta(line, theta):
    new_endpoint = calculate_rotated_line(line[0], line[1], theta)
    new_line = [line[1], new_endpoint]
    
    plt.plot([line[0][0], line[1][0]], [line[0][1], line[1][1]], 'r-', label='Original Line')

    # Plot the new line
    plt.plot([new_line[0][0], new_line[1][0]], [new_line[0][1], new_line[1][1]], 'b-', label='New Line')
    
    # Plotting points for clarity
    plt.scatter(*line[0], color='red', zorder=5)
    plt.scatter(*line[1], color='red', zorder=5)
    plt.scatter(*new_line[0], color='blue', zorder=5)
    plt.scatter(*new_line[1], color='blue', zorder=5)
    
    # Labeling the points
    plt.text(line[0][0], line[0][1], 'A', fontsize=12, ha='right')
    plt.text(line[1][0], line[1][1], 'B', fontsize=12, ha='right')
    plt.text(new_line[1][0], new_line[1][1], "B'", fontsize=12, ha='right')
    
    plt.xlim(-1, 2)
    plt.ylim(-1, 2)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.grid(True)
    plt.legend()
    plt.title('Visualizing Line Transformation')
    plt.show()


In [ ]:
# a bit dirty. A good way to redraw
def redraw_initial_sketch():
    fig, ax = plt.subplots()
    ax.set_xlim(-30, 80)
    ax.set_ylim(-20, 70)
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    
    angle_count_tuples = [[0, 60], [10, 9], [0, 20], [10, 9], [0, 60], [10, 9], [0, 20], [10, 9]]
    rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
    init_point_sequence = [[0, 5], [1, 5]]
    point_sequence_one = generate_point_sequence(init_point_sequence, rotation_angles)
    ax = draw_lines_based_on_points_on_plot(point_sequence_one, ax, line_color='blue')
    
    angle_count_tuples = [[0, 60], [6, 15], [0, 20], [6, 15], [0, 60], [6, 15], [0, 20], [6, 15]]
    rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
    init_point_sequence = [[-0.5, 1], [0.5, 1]]
    point_sequence_two = generate_point_sequence(init_point_sequence, rotation_angles)
    ax = draw_lines_based_on_points_on_plot(point_sequence_two, ax, line_color='red')

    return fig, ax

### Theory

In [ ]:
html(calculate_rotated_line)

In [ ]:
line = [[0, 0], [0.7071, 0.7071]]
theta = np.radians(0)
visualize_line_change_by_theta(line, theta)

In [ ]:
line = [[0, 0], [1, 0]]
theta = np.radians(10)
visualize_line_change_by_theta(line, theta)

### Streets-Building

In [ ]:
def calc_roation_angles_from_tuple(angle_count_tuples):
    return np.radians(np.repeat([angle for angle, count in angle_count_tuples], [count for angle, count in angle_count_tuples]))

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(-30, 80)
ax.set_ylim(-20, 70)
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

angle_count_tuples = [[0, 60], [10, 9], [0, 20], [10, 9], [0, 60], [10, 9], [0, 20], [10, 9]]
rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
init_point_sequence = [[0, 5], [1, 5]]
point_sequence_one = generate_point_sequence(init_point_sequence, rotation_angles)
ax = draw_lines_based_on_points_on_plot(point_sequence_one, ax, line_color='blue')

angle_count_tuples = [[0, 60], [6, 15], [0, 20], [6, 15], [0, 60], [6, 15], [0, 20], [6, 15]]
rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
init_point_sequence = [[-0.5, 1], [0.5, 1]]
point_sequence_two = generate_point_sequence(init_point_sequence, rotation_angles)
ax = draw_lines_based_on_points_on_plot(point_sequence_two, ax, line_color='red')

plt.show()

### Car-Simulation

In [ ]:
scatter_one = ax.scatter([], [], c='green', s=10)
scatter_two = ax.scatter([], [], c='orange', s=10)
time_scale = 5

def update(i):
    global ax, scatter_one, scatter_two, time_scale, point_sequence_one, point_sequence_two
    
    points_indices_one = np.mod(time_scale * np.array([2*i, i+20, i+50]), point_sequence_one.shape[0])
    points_indices_two = np.mod(time_scale * np.array([i+5, i+30, i+50]), point_sequence_two.shape[0])
    
    points_one = np.array([point_sequence_one[index] for index in points_indices_one])
    points_two = np.array([point_sequence_two[index] for index in points_indices_two])
    
    scatter_one.set_offsets(points_one)
    scatter_two.set_offsets(points_two)
    
    return scatter_one, scatter_two

ani = FuncAnimation(fig, update, frames=range(200), repeat=True, blit=True)
plt.close(fig)
HTML(ani.to_html5_video())

In [ ]:
N = 1000000
time_scale = 1
indices = np.arange(N)

indices_1 = np.mod(time_scale * np.array([2*indices, indices+20, indices+50]), point_sequence_one.shape[0])
indices_2 = np.mod(time_scale * np.array([indices+5, indices+30, indices+50]), point_sequence_two.shape[0])

points_one = point_sequence_one[indices_1]
points_two = point_sequence_two[indices_2]

#### Speed-Up traffic-simulation

In [ ]:
# # lane-id, lane-idx, speed, prob.-lane-change, unique-id
# cars = torch.tensor([[0, 1, 4, 0.1, 1], [0, 2, 3, 0.1, 2], [0, 3, 5, 0.1, 3], [1, 4, 3, 0.1, 4], [1, 5, 7, 0.1, 5], [0, 6, 3, 0.1, 6], [1, 10, 1, 0.1, 7], [1, 11, 3, 0.1, 8], [0, 13, 5, 0.1, 9], [0, 14, 3, 0.1, 10]], dtype=torch.float)
# cars = sort_matrix_by_nth_and_mth_column(cars)

In [ ]:
car_count = 50
len_track = point_sequence_one.shape[0]
cars_track = torch.randint(0, 2, (car_count,))
cars_position = torch.arange(0, len_track)[torch.randint(0, len_track, (car_count,))]
cars_speed = torch.randint(1, 10, (car_count,))
chance_for_takeover = torch.rand(car_count) / 4
cars_id = torch.arange(0, car_count)
cars = torch.column_stack([cars_track, cars_position, cars_speed, chance_for_takeover, cars_id])
cars = sort_matrix_by_nth_and_mth_column(cars)
# cars

In [ ]:
x_length = 100
fig, ax = plt.subplots()
ax.set_xlim(-20, x_length + 20)
ax.set_ylim(-10, x_length + 30)
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

angle_count_tuples = [[0, x_length], [10, 9], [0, x_length], [10, 9], [0, x_length], [10, 9], [0, x_length], [10, 9]]
rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
init_point_sequence = [[0, 5], [1, 5]]
point_sequence_one = generate_point_sequence(init_point_sequence, rotation_angles)
ax = draw_lines_based_on_points_on_plot(point_sequence_one, ax, line_color='blue')

angle_count_tuples = [[0, x_length], [6, 15], [0, x_length], [6, 15], [0, x_length], [6, 15], [0, x_length], [6, 15]]
rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
init_point_sequence = [[-0.5, 1], [0.5, 1]]
point_sequence_two = generate_point_sequence(init_point_sequence, rotation_angles)
ax = draw_lines_based_on_points_on_plot(point_sequence_two, ax, line_color='red')

plt.show()

In [ ]:
# both tracks must be the same length, otherwise lane-changes would be more complicated (not implemented)
track_sizes = torch.tensor([[0., point_sequence_one.shape[0]], [1., point_sequence_one.shape[0]]])
iterations = 500

points_indices_one_list = []
points_indices_two_list = []

# visualize updates
for _ in range(iterations):
    proportional_difference_lanes = point_sequence_two.shape[0] / point_sequence_one.shape[0]

    cars = update_cars(cars, track_sizes)
    cars_on_lane_one = cars[cars[:, 0] == 0]
    cars_on_lane_two = cars[cars[:, 0] == 1]

    points_indices_one_list.append(torch.remainder(cars_on_lane_one[:, 1], point_sequence_one.shape[0]))
    points_indices_two_list.append(torch.remainder(proportional_difference_lanes * cars_on_lane_two[:, 1], point_sequence_two.shape[0]))

In [ ]:
scatter_one = ax.scatter([], [], c='green', s=10)
scatter_two = ax.scatter([], [], c='orange', s=10)

def update(i):
    global ax, scatter_one, scatter_two, point_sequence_one, point_sequence_two

    if points_indices_one_list[i].shape[0] != 0:
        points_indices_one = points_indices_one_list[i]
        points_one = np.array([point_sequence_one[int(index)] for index in points_indices_one])
        scatter_one.set_offsets(points_one)

    if points_indices_two_list[i].shape[0] != 0:
        points_indices_two = points_indices_two_list[i]
        points_two = np.array([point_sequence_two[int(index)] for index in points_indices_two])
        scatter_two.set_offsets(points_two)
    
    return scatter_one, scatter_two

ani = FuncAnimation(fig, update, frames=range(iterations), repeat=True, blit=True)
plt.close(fig)
HTML(ani.to_html5_video())